In [34]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [35]:
pop = pd.read_csv('data/시군구_전처리완료.csv', encoding='euc-kr')
print(pop.shape)
pop.head()

(250, 10)


,광역시도,시군구,20-39세계,20-39세남자,20-39세여자,65세이상계,인구수계,소멸비율,소멸위기지역,소멸위기고위험지역
0,강원도,강릉시,47006,25199,21807,43830,211643,0.4975,True,False
1,강원도,고성군,5217,3175,2042,7755,26792,0.2633,True,False
2,강원도,동해시,18727,10469,8258,17572,89814,0.4700,True,False
3,강원도,삼척시,13583,7469,6117,16077,65623,0.3805,True,False
4,강원도,속초시,18080,9909,8171,15170,81497,0.5386,False,False


- 고유 ID 만들기

In [36]:
len(pop.시군구.unique())        # pop.shape와 값이 다르다 즉 중복이 있다. 250 != 226

226

In [37]:
# 시군구 값을 기본값으로 가져가는데
# ex) 고성군 - 고성(강원), 고성(경남)
# ex) 광역시 - 서울 용산, 서울 서대문, 대전 서구, 대전 유성, 세종
# ex) 행정구 - 수원 장안, 용인 수지, 고양 일산동, 창원 합포, 창원 회원
tmp_gu_dict = {
    '수원': ['장안구', '권선구', '팔달구', '영통구'],
    '성남': ['수정구', '중원구', '분당구'],
    '안양': ['만안구', '동안구'],
    '안산': ['상록구', '단원구'],
    '고양': ['덕양구', '일산동구', '일산서구'],
    '용인': ['처인구', '기흥구', '수지구'],
    '청주': ['상당구', '서원구', '흥덕구', '청원구'],
    '천안': ['동남구', '서북구'],
    '전주': ['완산구', '덕진구'],
    '포항': ['남구', '북구'],
    '창원': ['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
}

In [38]:
metro_list = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '대전광역시', '광주광역시', '울산광역시']
si_name = [None] * len(pop)

In [39]:
for i in pop.index:
    if pop.광역시도[i] in metro_list:
        if len(pop.시군구[i]) == 2:
            si_name[i] = pop.광역시도[i][:2] + ' ' + pop.시군구[i]      # 서울 중구, 대전 서구
        else:
            si_name[i] = pop.광역시도[i][:2] + ' ' + \
                pop.시군구[i][:-1]      # 서울 용산, 서울 서대문
    else:
        if pop.시군구[i][:-1] == '고성':
            if pop.광역시도[i] == '강원도':
                si_name[i] = '고성(강원)'
            else:
                si_name[i] = '고성(경남)'
        else:
            si_name[i] = pop.시군구[i][:-1]         # 세종, 광명, 김포, 강릉

        for key, value in tmp_gu_dict.items():
            if pop.시군구[i] in value:
                if len(pop.시군구[i]) == 2:
                    si_name[i] = key + ' ' + pop.시군구[i]      # 포항 북구, 포항 남구
                elif len(pop.시군구[i]) == 5:
                    si_name[i] = key + ' ' + pop.시군구[i][2:-1]    # 창원 합포, 창원 회원
                else:
                    si_name[i] = key + ' ' + pop.시군구[i][:-1]     # 수원 팔달, 용인 수지

In [40]:
pop['ID'] = si_name
pop.shape

(250, 11)

In [41]:
# ID를 인덱스로 변환
pop.set_index('ID', inplace=True)
pop.head()

,광역시도,시군구,20-39세계,20-39세남자,20-39세여자,65세이상계,인구수계,소멸비율,소멸위기지역,소멸위기고위험지역
ID,,,,,,,,,,
강릉,강원도,강릉시,47006,25199,21807,43830,211643,0.4975,True,False
고성(강원),강원도,고성군,5217,3175,2042,7755,26792,0.2633,True,False
동해,강원도,동해시,18727,10469,8258,17572,89814,0.4700,True,False
삼척,강원도,삼척시,13583,7469,6117,16077,65623,0.3805,True,False
속초,강원도,속초시,18080,9909,8171,15170,81497,0.5386,False,False


In [42]:
pop.to_csv('data/시군구_ID부여.csv', encoding='euc-kr')